In [ ]:
import nltk
import spacy

import pandas as pd
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from sklearn.feature_extraction.text import CountVectorizer



In [ ]:
# carrega os Tweets classificados
dfTweetsAux = pd.read_csv('dados/tweets_classificados.txt')

In [ ]:
# Carrega StopWords
nlp = spacy.load('pt_core_news_sm')
nltk.download('stopwords')

# stopwords NLTK
stops_nltk = nltk.corpus.stopwords.words('portuguese')

# stopwords SpaCy
stops_spacy = nlp.Defaults.stop_words

# stopwords do SpaCy e NLTK combinadas
stops = list(set(stops_spacy).union(set(stops_nltk)))
len(stops)

In [ ]:
# função de lematização completa do documento
def fn_lematiza_texto(texto):
    sent = []
    doc = nlp(texto)
    for word in doc:
        sent.append(word.lemma_)
        
    return ' '.join(sent)

# função de lematização dos verbos do documento
def fn_lematiza_verb_texto(texto):
    sent = []
    doc = nlp(texto)
    for word in doc:
        if word.pos_ =='VERB':
            sent.append(word.lemma_)
        else:
            sent.append(word.text)
    return " ".join(sent)


# função para limpar documento
def fn_limpa_texto(texto):
    texto = re.sub(r'@\w+','',texto)
    texto = re.sub(r'#','',texto)
    texto = re.sub(r'RT[\s]+','',texto)  
    #texto = re.sub(r'rt[\s]+','',texto)  
    texto = re.sub(r'https:/\/\S+','',texto)
    texto = re.sub(r'[,.:;]','',texto)
    texto = re.sub(r'\n',' ',texto)
    texto = re.sub(u'[^a-zA-ZâêîôûÂÊÎÔÛáéíóúÁÉÍÓÚàèìòùÀÈÌÒÙãõÃÕçÇ ]', '', texto)
    texto = texto.strip().lower()
    
    return texto


In [ ]:
# aplica normalização no dataframe
dfTweetsAux['texto_trat'] = dfTweetsAux.texto.apply(fn_limpa_texto)

# aplica a lematização no dataframe
dfTweetsAux['texto_lemma'] = dfTweetsAux.texto_trat.apply(fn_lematiza_texto)

# aplica a lematização dos verbos no dataframe
dfTweetsAux['texto_lemma_verb'] = dfTweetsAux.texto_trat.apply(fn_lematiza_verb_texto)

# apaga tweets nulos
dfTweetsAux.dropna(inplace=True)

In [ ]:
# analisa alguns dados
print(dfTweetsAux['texto'][1])
print(dfTweetsAux['texto_trat'][1])
print(dfTweetsAux['texto_lemma'][1])
print(dfTweetsAux['texto_lemma_verb'][1])

In [ ]:
# Modelo final

# vetorização tf-idf com combinação de unigrama e bigrama
vetor = TfidfVectorizer(ngram_range=(1,2), use_idf=True) 

# texto tratado e verbos lematizado
vetor.fit(dfTweetsAux.texto_lemma_verb)
text_vect = vetor.transform(dfTweetsAux.texto_lemma_verb)

# separa as amostras de Treino (80%) e Texte (20%)
X_train,X_test,y_train,y_test = train_test_split(
        text_vect, 
        dfTweetsAux['sentimento'], 
        test_size = 0.2,
        random_state = 42
    )

# define e treina o modelo de classificação
modelo = SVC(kernel="rbf", C=5, class_weight="balanced")

modelo.fit(X_train, y_train)

# valida a parformance
y_prediction = modelo.predict(X_test)
accuracy = accuracy_score(y_prediction, y_test)

print(text_vect.shape)
print(accuracy)


In [ ]:
# salva modelo e vetor de bag of words
import pickle

pickle.dump(vetor, open('modelo_verorizacao.pkl', 'wb'))
pickle.dump(modelo, open('modelo_classificacao.pkl', 'wb'))